# Twitter Scrape

Traversing a whole lot of shit...

In [1]:
# Libraries
import pandas as pd
import numpy as np
import sys, os
from time import time

from sklearn.externals import joblib

from multiprocessing import Pool

In [2]:
# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

from ldig.predict_tweet import *
detector = Detector('../src/ldig/models/model.latin/')

In [3]:
# Directories
ext_dir = '../data/external/'
proc_dir = '../data/processed/'

scrape_in = ext_dir + 'scrape/'
scrape_out = proc_dir + 'scrape/{}-extract.csv'

In [4]:
# cols to keep
cols_scrape = [
    'username',
    'longitude',
    'latitude',
    'date',
    'message',
    'tweetID',
    'language',
    'hashtag1',
    'hashtag2',
    'hashtag3',
    'hashtag4',
    'hashtag5'
]
cols_out = [
    'username',
    'longitude',
    'latitude',
    'date',
    'message',
    'hashtag1',
    'hashtag2',
    'hashtag3',
    'hashtag4',
    'hashtag5'
]

In [5]:
# Classifier
clf = joblib.load('../data/processed/class/3-28/model.pkl')

# to prepare us for processing raw tweets
%run ../src/twitter-sentiment/preprocessing/__init__.py

In [6]:
# List of all the files we'll be evaluating
scrape_f = os.listdir(scrape_in)

In [7]:
def process_cleaning(tweet):
    try:
        return process_one(tweet)
    except:
        #print (tweet)
        return None

def process_stemming(tweet):
    try:
        return stem_one(tweet)
    except:
        #print (tweet)
        return None
    
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, 10)
    pool = Pool(4)
    #df = pd.concat(pool.map(func, df_split))
    df = [pool.map(func, df_split)]
    pool.close()
    pool.join()
    return df

def parallelize_series(series, func):
    pool = Pool(6)

    df = pool.map(func, series)

    pool.close()
    pool.join()
    return df

In [28]:
def process_scrape_csv(csv_f, verbose=False):
    """
    """
    if (verbose):
        # Set a pattern for verbosity
        metrics = {'f_name': csv_f}
        start = time()
        print (csv_f)
    
    # read in
    f_ = csv_f
    df = pd.read_csv(f_, usecols=cols_scrape, dtype={'message': str})
    if (verbose):
        metrics['orig'] = df.shape[0]
        print ("Number of tweets\n\tOriginal:\t\t{}".format(metrics['orig']))
        
    # pre screen our messages
    df.dropna(subset=['message'], inplace=True)
    if (verbose):
        metrics['message'] = df.shape[0]
        print ("\tWith messages:\t\t{}".format(metrics['message']))
        
    # filter, keep only english tweets
    df = df[df.language == 'en']
    if (verbose):
        metrics['eng'] = df.shape[0]
        print ("\tLang (twit.) eng.:\t{}".format(metrics['eng']))
    
    # Predict language
    ## Tested and it doesn't help...
    #df['pred_lang'] = parallelize_series(df.message.values, detector.get_tweet_lang)
    #english = df[df.pred_lang == 'en']
    #if (verbose):
        #print ("\tLang (class.) eng.:\t\t\t{}".format(df.shape[0]))
    
    # tokenize
    df['clean'] = parallelize_series(df.message.values, process_cleaning)
    df = df[pd.notnull(df['clean'])]
    if (verbose):
        metrics['clean'] = df.shape[0]
        print ("\tWith viable tokens:\t{}".format(metrics['clean']))
        
    # stem
    df['stem'] = parallelize_series(df.clean.values, process_stemming)
    df = df[pd.notnull(df['stem'])]
    if (verbose):
        metrics['stem'] = df.shape[0]
        print ("\tWith stems:\t\t{}".format(metrics['stem']))
    
    # CLASSIFY
    df['label'] = clf.predict(df.stem.values)
    df = df[df.label == 1]
    if (verbose):
        metrics['label'] = df.shape[0]
        print ("\tClassified+:\t\t{}".format(metrics['label']))
        
    #if verbose, lets log that out
    if (verbose):
        secs = time() - start
        metrics['time'] = secs
        print ("... in {} seconds\n".format(secs))
        
        log.append(metrics)
    
    return df

In [29]:
# Keep a dataframe for a log
#log_df = pd.DataFrame(columns=['f_name', 'time', 'orig', 'message', 'eng', 'clean', 'stem', 'label'])
log = []

In [30]:
for f in scrape_f:
    
    global log_df
    # read in new tweets
    labeled_df = process_scrape_csv((scrape_in + f), verbose=True)
    
    # format the new file name, getting rid of '.csv'
    out_f = scrape_out.format(f[:-4])
    
    # write out the now filtered csv
    #labeled_df.to_csv(out_f, index=False)
    
    lat_f = out_f[:-4] + '-gps.csv'
    #labeled_df[pd.notnull(labeled_df['longitude'])].to_csv(lat_f, columns=cols_out, index=False)

../data/external/scrape/tweets_immigrant_34316.csv
Number of tweets
	Original:		375600
	With messages:		374678
	Lang (twit.) eng.:	180172
	With viable tokens:	180172
	With stems:		180170
	Classified+:		2341
... in 22.337826251983643 seconds

../data/external/scrape/tweets_immigrant_34315.csv
Number of tweets
	Original:		274900
	With messages:		274215
	Lang (twit.) eng.:	141850
	With viable tokens:	141850
	With stems:		141849
	Classified+:		1483
... in 17.7563316822052 seconds



In [31]:
log

[{'clean': 180172,
  'eng': 180172,
  'f_name': '../data/external/scrape/tweets_immigrant_34316.csv',
  'label': 2341,
  'message': 374678,
  'orig': 375600,
  'stem': 180170,
  'time': 22.337826251983643},
 {'clean': 141850,
  'eng': 141850,
  'f_name': '../data/external/scrape/tweets_immigrant_34315.csv',
  'label': 1483,
  'message': 274215,
  'orig': 274900,
  'stem': 141849,
  'time': 17.7563316822052}]

In [27]:
pd.DataFrame.from_records(log_df)

,clean,eng,f_name,label,message,orig,stem,time
0,180172,180172,../data/external/scrape/tweets_immigrant_34316...,2341,374678,375600,180170,22.011650
1,141850,141850,../data/external/scrape/tweets_immigrant_34315...,1483,274215,274900,141849,17.965607
